In [4]:
import cirq
import numpy as np

# Define the unitary matrix
matrix = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

# Create the MatrixGate from the unitary matrix
matrix_gate = cirq.MatrixGate(matrix)

# Create the qubits
qubits = cirq.LineQubit.range(2)

# Create the circuit
circuit = cirq.Circuit()

# Apply the MatrixGate to the qubits
circuit.append(matrix_gate.on(*qubits))

# Simulate the circuit
simulator = cirq.Simulator()

# Run the simulation and print the results
print("Circuit:")
print(circuit)

result = simulator.simulate(circuit)
print("\nFinal state vector:")
print(result.final_state_vector)


Circuit:
      ┌       ┐
      │0 1 0 0│
0: ───│1 0 0 0│───
      │0 0 1 0│
      │0 0 0 1│
      └       ┘
      │
1: ───#2──────────

Final state vector:
[0.+0.j 1.+0.j 0.+0.j 0.+0.j]


In [1]:
import cirq
import numpy as np



# Function to create an oracle gate from a given unitary matrix
def create_oracle_gate(oracle_matrix):
    
    return cirq.MatrixGate(oracle_matrix)

# Function to prompt the user to input a 4x4 unitary matrix
def input_unitary_matrix():
    print("Enter the 16 comma-separated values for the 4x4 unitary matrix:")
    #matrix_values = input().strip().split(',')
    matrix_values = "1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1".split(',')
    oracle_matrix = np.array([[float(matrix_values[i]) for i in range(4)],
                              [float(matrix_values[i]) for i in range(4, 8)],
                              [float(matrix_values[i]) for i in range(8, 12)],
                              [float(matrix_values[i]) for i in range(12, 16)]])
    return oracle_matrix

# Define Deutsch's algorithm
def deutsch_algorithm(oracle_gate):
    # Define qubits
    qubits = cirq.LineQubit.range(2)

    # Create a quantum circuit
    circuit = cirq.Circuit()

    # Apply Hadamard gate to both qubits to create superposition
    circuit.append(cirq.X(qubits[1]))
    circuit.append([cirq.H(q) for q in qubits])

    # Apply the oracle gate
    circuit.append(oracle_gate(*qubits[:2]))

    # Apply Hadamard gate to the first qubit only
    circuit.append(cirq.H(qubits[0]))

    # Measure the first qubit
    circuit.append(cirq.measure(qubits[0], key='result'))

    return circuit

# Prompt the user to input the 4x4 unitary matrix
oracle_matrix = input_unitary_matrix()

# Create the oracle gate from the user-defined unitary matrix
oracle_gate = create_oracle_gate(oracle_matrix)

# Run Deutsch's algorithm with the user-defined oracle gate
circuit = deutsch_algorithm(oracle_gate)

simulator = cirq.Simulator()
measurement_outcomes = []

for _ in range(10):
    result = simulator.run(circuit)
    measurement_outcome = result.measurements['result'][0][0]
    measurement_outcomes.append(measurement_outcome)

# Print all measurement outcomes
print("Measurement outcomes:", measurement_outcomes)

# Determine the type of function based on the measurement outcomes
function_type = "constant" if all(outcome == 0 for outcome in measurement_outcomes) else "balanced"
print("Function type:", function_type)

# Print the circuit
print("\nCircuit:")
print(circuit)


Enter the 16 comma-separated values for the 4x4 unitary matrix:
Measurement outcomes: [np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0)]
Function type: constant

Circuit:
              ┌           ┐
              │1. 0. 0. 0.│
0: ───H───────│0. 1. 0. 0.│───H───M('result')───
              │0. 0. 1. 0.│
              │0. 0. 0. 1.│
              └           ┘
              │
1: ───X───H───#2────────────────────────────────


In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import numpy as np
from qiskit.circuit.library import UnitaryGate

# --- Build the 8x8 oracle matrix for f(x1, x2) = x1 XOR x2 ---
oracle_matrix = np.eye(8)

# Basis ordering: |x1, x2, y>
# Flip output qubit when x1 != x2

# Case 01 -> flip y
oracle_matrix[2,2] = 0
oracle_matrix[2,3] = 1
oracle_matrix[3,3] = 0
oracle_matrix[3,2] = 1

# Case 10 -> flip y
oracle_matrix[4,4] = 0
oracle_matrix[4,5] = 1
oracle_matrix[5,5] = 0
oracle_matrix[5,4] = 1

# Wrap in a gate
oracle_gate = UnitaryGate(oracle_matrix, label="XOR Oracle")

# --- Deutsch–Jozsa algorithm for 2 input qubits ---
def deutsch_jozsa(oracle):
    qc = QuantumCircuit(3, 2)  # 2 input + 1 output

    # Step 1: Initialize output qubit to |1>
    qc.x(2)

    # Step 2: Hadamard on all qubits
    qc.h([0,1,2])

    # Step 3: Apply oracle
    qc.append(oracle, [0,1,2])

    # Step 4: Hadamard on input qubits only
    qc.h([0,1])

    # Step 5: Measure input qubits
    qc.measure([0,1], [0,1])

    return qc

qc = deutsch_jozsa(oracle_gate)

# --- Simulate ---
simulator = Aer.get_backend("qasm_simulator")
compiled = transpile(qc, simulator)
result = simulator.run(compiled, shots=1024).result()
counts = result.get_counts()

print("Measurement outcomes:", counts)
print("\nCircuit:")
print(qc.draw())


Measurement outcomes: {'00': 1024}

Circuit:
     ┌───┐     ┌─────────────┐┌───┐┌─┐   
q_0: ┤ H ├─────┤0            ├┤ H ├┤M├───
     ├───┤     │             │├───┤└╥┘┌─┐
q_1: ┤ H ├─────┤1 XOR Oracle ├┤ H ├─╫─┤M├
     ├───┤┌───┐│             │└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤2            ├──────╫──╫─
     └───┘└───┘└─────────────┘      ║  ║ 
c: 2/═══════════════════════════════╩══╩═
                                    0  1 


In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import numpy as np
from qiskit.circuit.library import UnitaryGate

# --- Build the 8x8 unitary matrix manually ---
oracle_matrix = np.eye(8)  # start with identity

# Flip output (last qubit) if the first input qubit is 1
# Basis ordering in Qiskit: |q0 q1 q_out>
# We want: |10,0> -> |10,1> ; |10,1> -> |10,0>
#          |11,0> -> |11,1> ; |11,1> -> |11,0>

oracle_matrix[4,4] = 0
oracle_matrix[4,5] = 1
oracle_matrix[5,5] = 0
oracle_matrix[5,4] = 1

oracle_matrix[6,6] = 0
oracle_matrix[6,7] = 1
oracle_matrix[7,7] = 0
oracle_matrix[7,6] = 1

# Wrap in a gate
oracle_gate = UnitaryGate(oracle_matrix, label="Balanced Oracle")

# --- Deutsch–Jozsa algorithm for 2 input qubits ---
def deutsch_jozsa(oracle):
    qc = QuantumCircuit(3, 2)  # 2 input qubits + 1 output qubit

    # Step 1: Initialize output qubit to |1>
    qc.x(2)

    # Step 2: Hadamard on all qubits
    qc.h([0,1,2])

    # Step 3: Apply oracle
    qc.append(oracle, [0,1,2])

    # Step 4: Hadamard on input qubits only
    qc.h([0,1])

    # Step 5: Measure input qubits
    qc.measure([0,1], [0,1])

    return qc

qc = deutsch_jozsa(oracle_gate)

# --- Simulate ---
simulator = Aer.get_backend("qasm_simulator")
compiled = transpile(qc, simulator)
result = simulator.run(compiled, shots=1024).result()
counts = result.get_counts()

print("Measurement outcomes:", counts)
print("\nCircuit:")
print(qc.draw())


Measurement outcomes: {'00': 1024}

Circuit:
     ┌───┐     ┌──────────────────┐┌───┐┌─┐   
q_0: ┤ H ├─────┤0                 ├┤ H ├┤M├───
     ├───┤     │                  │├───┤└╥┘┌─┐
q_1: ┤ H ├─────┤1 Balanced Oracle ├┤ H ├─╫─┤M├
     ├───┤┌───┐│                  │└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤2                 ├──────╫──╫─
     └───┘└───┘└──────────────────┘      ║  ║ 
c: 2/════════════════════════════════════╩══╩═
                                         0  1 


In [6]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer

# Deutsch–Jozsa algorithm (general n)
def deutsch_jozsa(n, oracle):
    """
    Build Deutsch–Jozsa circuit for n input qubits and 1 output qubit.
    oracle: a quantum circuit implementing Uf on (n+1) qubits
    """
    qc = QuantumCircuit(n+2, n)  # n inputs + 1 output, measure n inputs

    # Step 0: Initialize output qubit to |1>
    qc.x(n)

    # Step 1: Apply Hadamard to all qubits
    qc.h(range(n+1))

    # Step 2: Apply oracle Uf
    qc.compose(oracle, inplace=True)

    # Step 3: Apply Hadamard to input qubits only
    qc.h(range(n))

    # Step 4: Measure input qubits
    qc.measure(range(n), range(n))

    return qc


# --- Example: oracle for constant function f(x)=0 ---
def constant_oracle(n):
    qc = QuantumCircuit(n+1)
    # f(x)=0 means do nothing to output qubit
    return qc

# --- Example: oracle for balanced function f(x)=x1 (first input bit) ---
def balanced_oracle(n):
    qc = QuantumCircuit(n+1)
    qc.cx(0, n)  # control on first input qubit, target = output
    return qc


# Choose number of input qubits
n = 3  # for example, 3 input qubits

# Build oracle (try balanced)
oracle = balanced_oracle(n)

# Build Deutsch–Jozsa circuit
qc = deutsch_jozsa(n, oracle)
print(qc.draw())

# --- Run simulation ---
simulator = Aer.get_backend("qasm_simulator")
compiled = transpile(qc, simulator)
result = simulator.run(compiled, shots=1024).result()
counts = result.get_counts()

print("Measurement outcomes:", counts)


     ┌───┐          ┌───┐   ┌─┐
q_0: ┤ H ├───────■──┤ H ├───┤M├
     ├───┤┌───┐  │  └┬─┬┘   └╥┘
q_1: ┤ H ├┤ H ├──┼───┤M├─────╫─
     ├───┤├───┤  │   └╥┘ ┌─┐ ║ 
q_2: ┤ H ├┤ H ├──┼────╫──┤M├─╫─
     ├───┤├───┤┌─┴─┐  ║  └╥┘ ║ 
q_3: ┤ X ├┤ H ├┤ X ├──╫───╫──╫─
     └───┘└───┘└───┘  ║   ║  ║ 
q_4: ─────────────────╫───╫──╫─
                      ║   ║  ║ 
c: 3/═════════════════╩═══╩══╩═
                      1   2  0 
Measurement outcomes: {'001': 1024}
